# Raster Band Stacking with Rasterio and GDAL

In this notebook, I am demonstrating how to stack individual Landsat image bands into a multi-band GeoTIFF file using two popular geospatial Python libraries: **Rasterio** and **GDAL**. 

Band stacking is a common preprocessing step in remote sensing and GIS workflows that consolidates separate spectral bands into a single multi-band raster file. This makes it easier to manage and analyze multi-spectral data. When you download the images from USGS Earth Explorer, there are multiple bands downloaded for single scene. Users need to use different tools inside QGIS, ArcGIS to stack those bands together to create single image. This is an alternative to those options to stack the bands together.

We will:
- Automatically detect Landsat bands in a folder.
- Group bands belonging to the same scene.
- Stack bands in correct order.
- Save stacked multi-band GeoTIFF files using both Rasterio and GDAL.

---

### Libraries used and documentation links:
- Rasterio: https://rasterio.readthedocs.io/en/latest/  
  Key functions: `rasterio.open()`, `DatasetReader.read()`, `rasterio.open(..., 'w')`
- GDAL: https://gdal.org/python/  
  Key functions: `gdal.Open()`, `driver.Create()`, `RasterBand.WriteArray()`

---


In [7]:
"""
========================================================================================
Name: rasterio band stacker
Description: Automatically detects single or multiple Landsat images in a folder,
             stacks their bands into multi-band GeoTIFFs using Rasterio, and saves them
             with scene-based names in 'data/result_rasterio'.

Input:
    - Folder: data/images/
    - Files: Multiple *_SR_B*.TIF files from one or more scenes.

Output:
    - Folder: data/result_rasterio/
    - Stacked GeoTIFFs (one per scene) named like: <scene_id>_stacked_rasterio.tif

Author: Upendra Oli
========================================================================================
"""

import rasterio
import os
import re
from collections import defaultdict

def group_band_files(input_folder):
    """
    Groups TIF files in the specified input folder by their scene prefix.

    The grouping is based on the part of the filename before '_SR_B'.

    Args:
        input_folder (str): Path to the folder containing input band files.

    Returns:
        dict: A dictionary where keys are scene prefixes and values are lists of corresponding band filenames.
    """
    groups = defaultdict(list)
    for f in os.listdir(input_folder):
        if f.endswith('.TIF') and '_SR_B' in f:
            prefix = f.split('_SR_B')[0]
            groups[prefix].append(f)
    return groups

def stack_all_images_rasterio(input_subfolder='images', output_subfolder='result_rasterio'):
    """
    Stacks all bands of Landsat images found in the input folder into multi-band GeoTIFFs using Rasterio.

    It detects all scenes based on band filename prefixes, sorts bands numerically, stacks them,
    and writes the output GeoTIFFs to the specified result folder with appropriate naming.

    Args:
        input_subfolder (str): Subfolder inside 'data' containing input band files. Default is 'images'.
        output_subfolder (str): Subfolder inside 'data' where stacked images will be saved. Default is 'result_rasterio'.

    Returns:
        None. Saves stacked GeoTIFF files on disk.
    """
    # Set up full paths
    base_folder = 'data'
    input_folder = os.path.join(base_folder, input_subfolder)
    result_folder = os.path.join(base_folder, output_subfolder)
    os.makedirs(result_folder, exist_ok=True)

    # Group band files by image prefix
    grouped_files = group_band_files(input_folder)

    for prefix, files in grouped_files.items():
        # Sort bands numerically (e.g., B1, B2, B3...)
        band_files_sorted = sorted(files, key=lambda x: int(re.search(r'_SR_B(\d+)', x).group(1)))
        band_paths = [os.path.join(input_folder, f) for f in band_files_sorted]
        src_files = [rasterio.open(fp) for fp in band_paths]

        # Copy metadata from first band and update count
        meta = src_files[0].meta.copy()
        meta.update(count=len(src_files))

        # Create output file path
        output_path = os.path.join(result_folder, f"{prefix}_stacked_rasterio.tif")

        # Write stacked bands
        with rasterio.open(output_path, 'w', **meta) as dst:
            for idx, src in enumerate(src_files):
                dst.write(src.read(1), idx + 1)

        print(f"Rasterio: Saved stacked image to {output_path}")


if __name__ == "__main__":
    stack_all_images_rasterio()

Rasterio: Saved stacked image to data\result_rasterio\LC08_L2SP_141041_20250425_20250429_02_T1_stacked_rasterio.tif
Rasterio: Saved stacked image to data\result_rasterio\LC09_L2SP_141041_20250401_20250403_02_T1_stacked_rasterio.tif


---

### 2. GDAL Band Stacking
Here is the equivalent function using the GDAL library to perform the same task.

We open the bands, create a multi-band GeoTIFF, copy georeferencing info, and write each band to the output.

---

**Note:** GDAL uses different data type constants and API calls compared to Rasterio.



In [ ]:
"""
========================================================================================
Name: gdal band stacker
Description: Automatically detects single or multiple Landsat images in a folder,
             stacks their bands into multi-band GeoTIFFs using GDAL, and saves them
             with scene-based names in 'data/result_gdal'.

Input:
    - Folder: data/images/
    - Files: Multiple *_SR_B*.TIF files from one or more scenes.

Output:
    - Folder: data/result_gdal/
    - Stacked GeoTIFFs (one per scene) named like: <scene_id>_stacked_gdal.tif

Author: Upendra Oli
========================================================================================
"""

from osgeo import gdal
import os
import re
from collections import defaultdict

def group_band_files(input_folder):
    """
    Groups TIF files in the specified input folder by their scene prefix.

    The grouping is based on the part of the filename before '_SR_B'.

    Args:
        input_folder (str): Path to the folder containing input band files.

    Returns:
        dict: A dictionary where keys are scene prefixes and values are lists of corresponding band filenames.
    """
    groups = defaultdict(list)
    for f in os.listdir(input_folder):
        if f.endswith('.TIF') and '_SR_B' in f:
            prefix = f.split('_SR_B')[0]
            groups[prefix].append(f)
    return groups

def stack_all_images_gdal(input_subfolder='images', output_subfolder='result_gdal'):
    """
    Stacks all bands of Landsat images found in the input folder into multi-band GeoTIFFs using GDAL.

    It detects all scenes based on band filename prefixes, sorts bands numerically, stacks them,
    and writes the output GeoTIFFs to the specified result folder with appropriate naming.

    Args:
        input_subfolder (str): Subfolder inside 'data' containing input band files. Default is 'images'.
        output_subfolder (str): Subfolder inside 'data' where stacked images will be saved. Default is 'result_gdal'.

    Returns:
        None. Saves stacked GeoTIFF files on disk.
    """
    # Set up full paths
    base_folder = 'data'
    input_folder = os.path.join(base_folder, input_subfolder)
    result_folder = os.path.join(base_folder, output_subfolder)
    os.makedirs(result_folder, exist_ok=True)

    # Group band files by image prefix
    grouped_files = group_band_files(input_folder)

    for prefix, files in grouped_files.items():
        # Sort bands numerically (e.g., B1, B2, B3...)
        band_files_sorted = sorted(files, key=lambda x: int(re.search(r'_SR_B(\d+)', x).group(1)))
        band_paths = [os.path.join(input_folder, f) for f in band_files_sorted]

        # Open the first band as reference
        ref_ds = gdal.Open(band_paths[0])
        driver = gdal.GetDriverByName('GTiff')

        # Create output file path
        output_path = os.path.join(result_folder, f"{prefix}_stacked_gdal.tif")

        # Create multi-band TIFF
        out_ds = driver.Create(
            output_path,
            ref_ds.RasterXSize,
            ref_ds.RasterYSize,
            len(band_paths),
            gdal.GDT_UInt16
        )

        # Copy georeferencing and projection
        out_ds.SetGeoTransform(ref_ds.GetGeoTransform())
        out_ds.SetProjection(ref_ds.GetProjection())

        # Write each band
        for idx, band_path in enumerate(band_paths):
            band_data = gdal.Open(band_path).ReadAsArray()
            out_ds.GetRasterBand(idx + 1).WriteArray(band_data)

        out_ds.FlushCache()
        print(f"GDAL: Saved stacked image to {output_path}")


if __name__ == "__main__":
    stack_all_images_gdal()
